<a href="https://colab.research.google.com/github/CaseySobon/Recommendation-system/blob/main/Recommendation_for_music_selection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# innstall java
!apt-get install openjdk-8-jdk-headless -qq > /dev/null

# install spark (change the version number if needed)
!wget -q https://archive.apache.org/dist/spark/spark-3.2.1/spark-3.2.1-bin-hadoop3.2.tgz

# unzip the spark file to the current folder
!tar xf spark-3.2.1-bin-hadoop3.2.tgz

# set your spark folder to your system path environment.
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.2.1-bin-hadoop3.2"


# install findspark using pip
!pip install -q findspark

In [ ]:
import findspark

findspark.init()

from pyspark.sql import SparkSession

spark = SparkSession.builder.master("local[*]").getOrCreate()

from google.colab import files

In [ ]:
uploaded = files.upload()

Saving trainItem.data to trainItem.data


In [ ]:
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.recommendation import ALS
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.feature import OneHotEncoder, StringIndexer, VectorAssembler
from pyspark.sql.functions import min, max, col
from pyspark.sql import functions as F
from pyspark.sql.functions import lit
from pyspark.ml import Pipeline
import matplotlib.pyplot as plt
from pyspark.sql.types import IntegerType
import pandas as pd
import numpy as np

In [ ]:
training = spark.read.csv("trainItem.data", header = False)
training = training.withColumnRenamed("_c0", "userID").withColumnRenamed("_c1", "itemID").withColumnRenamed("_c2", "rating").orderBy('userID')

training = training.withColumn("userID", training["userID"].cast(IntegerType()))
training = training.withColumn("itemID", training["itemID"].cast(IntegerType()))
training = training.withColumn("rating", training["rating"].cast('float'))

print((training.count(), len(training.columns)))



(12403575, 3)


In [ ]:
als = ALS(
    maxIter=10,
    rank = 7,
    regParam=0.01,
    userCol="userID",
    itemCol="itemID",
    ratingCol="rating",
    nonnegative = True,
    implicitPrefs = False,
    coldStartStrategy="nan"
)


In [ ]:
model = als.fit(training)

In [ ]:
testing = spark.read.csv("testItem.data", header = False)
testing = testing.withColumnRenamed("_c0", "userID").withColumnRenamed("_c1", "itemID").withColumnRenamed("_c2", "rating").orderBy('rating',assending=True)
testing = testing.withColumn("userID", testing["userID"].cast(IntegerType()))
testing = testing.withColumn("itemID", testing["itemID"].cast(IntegerType()))
testing = testing.withColumn("rating", testing["rating"].cast('float'))


print((testing.count(), len(testing.columns)))


(120000, 3)


In [ ]:
predictions_track = model.transform(testing).orderBy('userID').fillna(20)
predictions_track = predictions_track.select('userID','itemID','prediction').withColumnRenamed('prediction','ptrack').withColumnRenamed('itemID','trackID')
predictions_track.show(16)

predictions_track.select('ptrack').describe().show()

+------+-------+---------+
|userID|trackID|   ptrack|
+------+-------+---------+
|199810|  74139| 84.71382|
|199810| 242681| 62.24705|
|199810|   9903| 67.05974|
|199810| 208019|62.093025|
|199810|  18515|   76.044|
|199810| 105760|61.343338|
|199812| 211361|110.85896|
|199812|  29189| 83.19683|
|199812| 142408|97.582375|
|199812| 223706| 89.01981|
|199812| 276940|37.979412|
|199812| 130023|103.94401|
|199813|  20968|23.017673|
|199813|  21571| 7.754678|
|199813| 188441|7.0513606|
|199813| 184173|27.387327|
+------+-------+---------+
only showing top 16 rows

+-------+-----------------+
|summary|           ptrack|
+-------+-----------------+
|  count|           120000|
|   mean|72.84750238832345|
| stddev|76.07917496596235|
|    min|              0.0|
|    max|        4818.1494|
+-------+-----------------+



In [ ]:
predictions.select('prediction').describe().show()

In [ ]:
x1 = pd.read_csv('testTrack_hierarchy.txt', sep='|', header=None, names=range(14),na_values='None',dtype=str).values

x2=pd.DataFrame(x1)
x3=x2.fillna(0)
x3[:] = x3[:].astype(int)
H_DF=spark.createDataFrame(x3).withColumnRenamed("0", "userID").withColumnRenamed("1", "trackID").withColumnRenamed("2", "albumID")\
.withColumnRenamed("3", "artistID").withColumnRenamed("4", "itemID3").withColumnRenamed("5", "itemID4").withColumnRenamed("6", "itemID5")\
.withColumnRenamed("7", "itemID6").withColumnRenamed("8", "itemID7").withColumnRenamed("9", "itemID8").withColumnRenamed("10", "itemID9")\
.withColumnRenamed("11", "itemID10").withColumnRenamed("12", "itemID11").withColumnRenamed("13", "itemID12")



In [ ]:
H_DF.show(6)

+------+-------+-------+--------+-------+-------+-------+-------+-------+-------+-------+--------+--------+--------+
|userID|trackID|albumID|artistID|itemID3|itemID4|itemID5|itemID6|itemID7|itemID8|itemID9|itemID10|itemID11|itemID12|
+------+-------+-------+--------+-------+-------+-------+-------+-------+-------+-------+--------+--------+--------+
|199810| 208019| 209288|       0|      0|      0|      0|      0|      0|      0|      0|       0|       0|       0|
|199810|  74139| 277282|  271146| 113360| 173467| 173655| 192976| 146792|  48505| 133159|       0|       0|       0|
|199810|   9903|      0|       0|  33722| 123396|  79926|  73523|      0|      0|      0|       0|       0|       0|
|199810| 242681| 190640|  244574|  61215|  17453| 274088|      0|      0|      0|      0|       0|       0|       0|
|199810|  18515| 146344|   33168|  19913|  48505| 154024|      0|      0|      0|      0|       0|       0|       0|
|199810| 105760|  93458|   11616| 131552| 173467|  48505| 133159

In [ ]:
testing_album = H_DF.select('userID','trackID','albumID').withColumn('rating',lit(0)).withColumnRenamed('albumID','itemID').orderBy('userID')

testing_album = testing_album.withColumnRenamed("_c0", "userID").withColumnRenamed("_c1", "itemID").withColumnRenamed("_c2", "rating").orderBy('rating',assending=True)
testing_album = testing_album.withColumn("userID", testing_album["userID"].cast(IntegerType()))
testing_album = testing_album.withColumn("trackID", testing_album["trackID"].cast(IntegerType()))
testing_album = testing_album.withColumn("itemID", testing_album["itemID"].cast(IntegerType()))
testing_album = testing_album.withColumn("rating", testing_album["rating"].cast('float'))
testing_album=testing_album.where("itemID!=0")
testing_album.show(3)

print((testing_album.count(), len(testing_album.columns)))

+------+-------+------+------+
|userID|trackID|itemID|rating|
+------+-------+------+------+
|199810| 208019|209288|   0.0|
|199810|  74139|277282|   0.0|
|199810| 242681|190640|   0.0|
+------+-------+------+------+
only showing top 3 rows

(111427, 4)


In [ ]:
predictions_album = model.transform(testing_album).orderBy('userID').fillna(20)
predictions_album = predictions_album.select('userID','trackID','prediction').withColumnRenamed('prediction','palbum')
predictions_album.show(16)

predictions_album.select('palbum').describe().show()

+------+-------+---------+
|userID|trackID|   palbum|
+------+-------+---------+
|199810|  74139| 83.42785|
|199810| 242681|61.945484|
|199810|  18515|74.734535|
|199810| 105760| 65.63859|
|199810| 208019|56.200005|
|199812| 211361| 38.04132|
|199812| 130023|102.65184|
|199812| 142408|101.67205|
|199812| 276940|21.353056|
|199812| 223706|87.489075|
|199812|  29189| 83.51416|
|199813| 188441| 28.22972|
|199813| 184173|28.558048|
|199813|  79640| 68.10348|
|199813|  21571|70.122856|
|199813| 111874|44.086304|
+------+-------+---------+
only showing top 16 rows

+-------+-----------------+
|summary|           palbum|
+-------+-----------------+
|  count|           111427|
|   mean|65.82174003635534|
| stddev|45.17700684568396|
|    min|              0.0|
|    max|        2952.9219|
+-------+-----------------+



In [ ]:
testing_artist = H_DF.select('userID','trackID','artistID').withColumn('rating',lit(0)).withColumnRenamed('artistID','itemID').orderBy('userID')

testing_artist = testing_artist.withColumnRenamed("_c0", "userID").withColumnRenamed("_c1", "itemID").withColumnRenamed("_c2", "rating").orderBy('rating',assending=True)
testing_artist = testing_artist.withColumn("userID", testing_artist["userID"].cast(IntegerType()))
testing_artist = testing_artist.withColumn("trackID", testing_artist["trackID"].cast(IntegerType()))
testing_artist = testing_artist.withColumn("itemID", testing_artist["itemID"].cast(IntegerType()))
testing_artist = testing_artist.withColumn("rating", testing_artist["rating"].cast('float'))
testing_artist=testing_artist.where("itemID!=0")
testing_artist.show(3)

print((testing_artist.count(), len(testing_artist.columns)))

+------+-------+------+------+
|userID|trackID|itemID|rating|
+------+-------+------+------+
|199810|  74139|271146|   0.0|
|199810| 242681|244574|   0.0|
|199810|  18515| 33168|   0.0|
+------+-------+------+------+
only showing top 3 rows

(109109, 4)


In [ ]:
predictions_artist = model.transform(testing_artist).orderBy(['userID']).fillna(20)
predictions_artist=predictions_artist.withColumnRenamed('prediction','partist').select('userID','trackID','partist')
predictions_artist.show(3)

predictions_artist.select('partist').describe().show()

+------+-------+--------+
|userID|trackID| partist|
+------+-------+--------+
|199810| 105760|73.41089|
|199810| 242681|64.58747|
|199810|  18515|76.67925|
+------+-------+--------+
only showing top 3 rows

+-------+-----------------+
|summary|          partist|
+-------+-----------------+
|  count|           109109|
|   mean|61.58091618657883|
| stddev|30.96084296299067|
|    min|              0.0|
|    max|        2388.3396|
+-------+-----------------+



In [ ]:
predict_matrix=predictions_track.join(predictions_album, ['userID','trackID'],'left').orderBy('userID').fillna(20)
predict_matrix=predict_matrix.join(predictions_artist, ['userID','trackID'],'left').orderBy('userID').fillna(20)

predict_matrix.show(6)
predict_matrix.select('palbum').describe().show()

+------+-------+---------+---------+---------+
|userID|trackID|   ptrack|   palbum|  partist|
+------+-------+---------+---------+---------+
|199810| 105760|61.343338| 65.63859| 73.41089|
|199810|   9903| 67.05974|     20.0|     20.0|
|199810|  74139| 84.71382| 83.42785|62.185196|
|199810| 208019|62.093025|56.200005|     20.0|
|199810|  18515|   76.044|74.734535| 76.67925|
|199810| 242681| 62.24705|61.945484| 64.58747|
+------+-------+---------+---------+---------+
only showing top 6 rows

+-------+------------------+
|summary|            palbum|
+-------+------------------+
|  count|            120000|
|   mean|62.548158558591375|
| stddev|45.104734050828085|
|    min|               0.0|
|    max|         2952.9219|
+-------+------------------+



In [ ]:
from pyspark.sql.functions import array, col, explode, lit, struct
from pyspark.sql import DataFrame
from typing import Iterable
def melt(H_DF: DataFrame, id_vars: Iterable[str], value_vars: Iterable[str], var_name: str="variable", value_name: str="value") -> DataFrame:
    """Convert :class:`DataFrame` from wide to long format."""

    # Create array<struct<variable: str, value: ...>>
    _vars_and_vals = array(*(struct(lit(c).alias(var_name), col(c).alias(value_name)) for c in value_vars))

    # Add to the DataFrame and explode
    _tmp = H_DF.withColumn("_vars_and_vals", explode(_vars_and_vals))

    cols = id_vars + [col("_vars_and_vals")[x].alias(x) for x in [var_name, value_name]]
    return _tmp.select(*cols)

melted_H = melt(H_DF,id_vars=['userID','trackID','albumID','artistID'],value_vars=['itemID3','itemID4','itemID5', 'itemID6','itemID7', 'itemID8','itemID9',\
                                                            'itemID10', 'itemID11','itemID12'])


melted_H=melted_H.where("value!=0")
melted_H=melted_H.withColumnRenamed("value", "itemID")


melted_H.select('itemID').describe().show()
melted_H.show(5)

+-------+------------------+
|summary|            itemID|
+-------+------------------+
|  count|            409994|
|   mean|132865.49718044655|
| stddev| 80155.78393975903|
|    min|              3969|
|    max|            294310|
+-------+------------------+

+------+-------+-------+--------+--------+------+
|userID|trackID|albumID|artistID|variable|itemID|
+------+-------+-------+--------+--------+------+
|199810|  74139| 277282|  271146| itemID3|113360|
|199810|  74139| 277282|  271146| itemID4|173467|
|199810|  74139| 277282|  271146| itemID5|173655|
|199810|  74139| 277282|  271146| itemID6|192976|
|199810|  74139| 277282|  271146| itemID7|146792|
+------+-------+-------+--------+--------+------+
only showing top 5 rows



In [ ]:
predictions_genre = model.transform(melted_H).orderBy(['userID','trackID']).fillna(20)
predictions_genre = predictions_genre.select('userID','trackID','prediction').withColumnRenamed('prediction','pgene')

predictions_genre.show(16)
predictions_genre.select('pgene').describe().show()

+------+-------+---------+
|userID|trackID|    pgene|
+------+-------+---------+
|199810|   9903|  46.0127|
|199810|   9903|53.750237|
|199810|   9903|44.256237|
|199810|   9903|59.234833|
|199810|  18515|27.577723|
|199810|  18515|78.993126|
|199810|  18515| 64.90471|
|199810|  74139|78.993126|
|199810|  74139|25.423084|
|199810|  74139|73.000725|
|199810|  74139| 73.78022|
|199810|  74139| 56.93692|
|199810|  74139| 48.85546|
|199810|  74139| 72.22985|
|199810| 105760| 72.22985|
|199810| 105760|73.000725|
+------+-------+---------+
only showing top 16 rows

+-------+-----------------+
|summary|            pgene|
+-------+-----------------+
|  count|           409994|
|   mean|57.53658174360858|
| stddev|30.13997244988061|
|    min|              0.0|
|    max|         2851.458|
+-------+-----------------+



In [ ]:
from pyspark.sql.functions import udf
from pyspark.sql.types import *

fn_l = [F.min,F.max,F.mean,F.variance,F.stddev]
col_l=['pgene']
expr = [fn(coln).alias(str(fn.__name__)+'_'+str(coln)) for fn in fn_l for coln in col_l]
tst_r = predictions_genre.groupby('userID','trackID').agg(*expr).fillna(0)
tst_r = tst_r.orderBy(['userID','trackID'])
tst_r.show(6)
tst_r.select('stddev_pgene').describe().show()

+------+-------+---------+---------+------------------+------------------+------------------+
|userID|trackID|min_pgene|max_pgene|        mean_pgene|    variance_pgene|      stddev_pgene|
+------+-------+---------+---------+------------------+------------------+------------------+
|199810|   9903|44.256237|59.234833| 50.81350135803223|  48.5295511768442|6.9663154663598315|
|199810|  18515|27.577723|78.993126| 57.15851910909017| 705.8885175940098|26.568562580501222|
|199810|  74139|25.423084|78.993126| 61.31705529349191| 364.3708759446474|19.088501144528017|
|199810| 105760| 67.53468|78.993126| 72.93959617614746|22.121927179410704| 4.703395282071315|
|199810| 242681| 38.34801| 48.20907|43.168103535970054| 24.34670108810436|4.9342376400113075|
|199812|  29189| 78.38157|100.42924|  88.9797757466634|61.277375997566104| 7.827986714191977|
+------+-------+---------+---------+------------------+------------------+------------------+
only showing top 6 rows

+-------+------------------+
|summa

In [ ]:
predict_matrix=predict_matrix.join(tst_r, ['userID','trackID'],'left').orderBy('userID').fillna(0)

predict_matrix.show(6)
predict_matrix.select('stddev_pgene').describe().show()

+------+-------+---------+---------+---------+---------+---------+------------------+------------------+------------------+
|userID|trackID|   ptrack|   palbum|  partist|min_pgene|max_pgene|        mean_pgene|    variance_pgene|      stddev_pgene|
+------+-------+---------+---------+---------+---------+---------+------------------+------------------+------------------+
|199810| 208019|62.093025|56.200005|     20.0|      0.0|      0.0|               0.0|               0.0|               0.0|
|199810|  74139| 84.71382| 83.42785|62.185196|25.423084|78.993126| 61.31705529349191| 364.3708759446474|19.088501144528017|
|199810| 242681| 62.24705|61.945484| 64.58747| 38.34801| 48.20907|43.168103535970054| 24.34670108810436|4.9342376400113075|
|199810| 105760|61.343338| 65.63859| 73.41089| 67.53468|78.993126| 72.93959617614746|22.121927179410704| 4.703395282071315|
|199810|  18515|   76.044|74.734535| 76.67925|27.577723|78.993126| 57.15851910909017| 705.8885175940098|26.568562580501222|
|199810|

In [ ]:
raw_df = spark.read.csv("merged_raw.csv", header = True)
raw_df.show(5)

+------+-------+---------+
|userID|trackID|Predictor|
+------+-------+---------+
|199810|   9903|      0.0|
|199810| 208019|      0.0|
|199810|  74139|     80.0|
|199810|  18515|     70.0|
|199810| 242681|      0.0|
+------+-------+---------+
only showing top 5 rows



In [ ]:
predict_matrix=predict_matrix.join(raw_df, ['userID','trackID'],'left').orderBy('userID').fillna(0)

In [ ]:
predict_matrix.show(12)

+------+-------+---------+---------+---------+---------+---------+------------------+------------------+------------------+---------+
|userID|trackID|   ptrack|   palbum|  partist|min_pgene|max_pgene|        mean_pgene|    variance_pgene|      stddev_pgene|Predictor|
+------+-------+---------+---------+---------+---------+---------+------------------+------------------+------------------+---------+
|199810|   9903| 67.05974|     20.0|     20.0|44.256237|59.234833| 50.81350135803223|  48.5295511768442|6.9663154663598315|      0.0|
|199810| 105760|61.343338| 65.63859| 73.41089| 67.53468|78.993126| 72.93959617614746|22.121927179410704| 4.703395282071315|     90.0|
|199810|  74139| 84.71382| 83.42785|62.185196|25.423084|78.993126| 61.31705529349191| 364.3708759446474|19.088501144528017|     80.0|
|199810| 208019|62.093025|56.200005|     20.0|      0.0|      0.0|               0.0|               0.0|               0.0|      0.0|
|199810| 242681| 62.24705|61.945484| 64.58747| 38.34801| 48.20

In [ ]:
predict_matrix.toPandas().to_csv('predict_matrix2.csv',index=False)